In [1]:
import k3d
import math
import numpy as np

# Y-Z flip
model_matrix = [
    1.0, 0.0, 0.0, 0.0,
    0.0, -1.0, 0.0, 0.0,
    0.0, 0.0, 1.0, 0.0,
    0.0, 0.0, 0.0, 1.0
]

texture = k3d.texture(open('assets/texture.png', 'br').read(), 'png', 
                      rotation=[math.radians(90),1,0,0], 
                      model_matrix = model_matrix,
                      name='Photo')

plot = k3d.plot()
plot += texture
plot.display()

Output()

In [2]:
texture.binary = open('assets/mandelbrot.jpg', 'br').read()
texture.name = 'Fractal'

In [3]:
texture.model_matrix = np.identity(4, dtype=np.float32)
texture.transform.rotation = np.zeros_like(texture.transform.rotation)

In [4]:
texture.puv = np.array([
    -5.0, 0.0, -5.0, # p
    10.0, 0.0,  0.0, # u
     0.0, 0.0, 10.0  # v
], dtype=np.float32)

In [5]:
plot.camera_reset()

In [6]:
from k3d.helpers import download
filename = download('https://github.com/FNNDSC/data/raw/master/nifti/adi_brain/adi_brain.nii.gz')

In [7]:
import nibabel as nib
nii_source = nib.load(filename)
img = nii_source.get_fdata()
dx, dy, dz = nii_source.header.get_zooms()
img = np.swapaxes(img,0,2)
nz, ny, nx = img.shape

In [8]:
from ipywidgets import widgets,interact

plot = k3d.plot()

bounds = [0, nx * dx, 0, ny * dy, 0, nz * dz]

texture = k3d.texture(attribute=img[0,:,:], 
                      color_map=k3d.basic_color_maps.Jet, 
                      color_range=[0.0, 1024.0],
                      bounds=bounds,
                      opacity_function=[-1,1,1,1],
                      name='Slice')
plot += texture

@interact(x=widgets.IntRangeSlider(value=[0, 1024], min=0, max=np.max(img), step=1, description='Color range:'))
def g(x):
    texture.color_range = x
    
@interact(z=widgets.IntSlider(value=0,min=0,max=img.shape[0]-1,step=1, description='Slice:'))
def g(z):
    texture.attribute =img[z, :, :]
    texture.transform.bounds = [bounds[0], bounds[1], bounds[2], bounds[3], -1 + z * dz, 1 + z * dz]

plot.display()

interactive(children=(IntRangeSlider(value=(0, 1024), description='Color range:', max=2012), Output()), _dom_c…

interactive(children=(IntSlider(value=0, description='Slice:', max=55), Output()), _dom_classes=('widget-inter…

Output()

In [10]:
data = plot.get_binary_snapshot()

with open('binary_snapshot.k3d', 'wb') as f:
    f.write(data)

In [9]:
plot.grid_auto_fit = False
plot.camera_auto_fit = False

In [ ]:
plot += k3d.marching_cubes(img.astype(np.float32), level=900, bounds=bounds, color=0xff0000)

In [ ]:
bounds = [0, nx * dx, 0, ny * dy, 0, nz * dz]

texture = k3d.texture(attribute=img[0,:,:], 
                      color_map=k3d.basic_color_maps.Jet, 
                      color_range=[0.0, 1024.0],
                      opacity_function=[-1,1,1,1],
                      bounds=bounds,
                      name='Slice')
plot += texture